In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor

import preprocess

# preprocess has get_processed_df and get_stratified_df

print("Successfully Loaded")

In [ ]:
df = preprocess.get_processed_df()
print(type(df))
print(df.shape)

In [ ]:
attribute_labels = df.columns[5:]

attributes = []
for a in attribute_labels:
    attributes.extend(list(df[a].values))

attributes = np.asarray([np.asarray(attributes)])
attributes = np.reshape(attributes, (len(attribute_labels), -1))
attributes = attributes.transpose()

labels = df['popularity'].values.reshape(-1, 1)

# Labels: popularity
# Attributes: features

print(attributes)
print(attributes.shape)
print(labels.shape)

In [ ]:
# Training random forest on whole dataframe

X_train, X_test, y_train, y_test = train_test_split(attributes, labels, test_size=0.2, random_state=0)
rand_forest = RandomForestRegressor(n_estimators = 100, criterion = 'mse', max_depth = None, min_samples_split = 2)

y_train = np.ravel(y_train)
y_test = np.ravel(y_test)
"""
The below line of code causes this: DataConversionWarning: A column-vector y was
passed when a 1d array was expected. Please change the shape of y to (n_samples,)

y_test = np.ravel(y_test)
"""
rand_forest.fit(X_train, y_train)

predictions = rand_forest.predict(X_test)
print('First 5 predictions: %s\n' % predictions[0:5])
print(y_test[:5])

print('Score: %s\n' % rand_forest.score(X_test, y_test))

In [ ]:
df_gen_dict = preprocess.get_stratified_df()
print(type(df_gen_dict))
print(df_gen_dict.keys())

In [ ]:
def get_attributes_labels(df):
    # remove genre, artist_name, track_name, track_id, popularity
    attribute_labels = df.columns[5:]

    attributes = []
    for a in attribute_labels:
        attributes.extend(list(df[a].values))

    attributes = np.asarray([np.asarray(attributes)])
    attributes = np.reshape(attributes, (len(attribute_labels), -1))
    attributes = attributes.transpose()

    labels = df['popularity'].values.reshape(-1, 1)
    
    return attributes, labels

In [ ]:
"""
Create dictionary. key: genre, value: RandomForestRegressor trained on the specific genre dataset
"""
reg_genre = dict()
for g in df_gen_dict.keys():
    attributes, labels = get_attributes_labels(df_gen_dict[g])
    X_train, X_test, y_train, y_test = train_test_split(attributes, labels, test_size=0.2, random_state=0)
    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)
    rand_forest = RandomForestRegressor(n_estimators = 100, criterion = 'mse', max_depth = None, min_samples_split = 2)
    rand_forest.fit(X_train, y_train)
    
    reg_genre[g] = rand_forest

    predictions = rand_forest.predict(X_test)
    print('Genre: ' + g)
    print('Score: %s\n' % rand_forest.score(X_test, y_test))
    